# Previsão de Preços de Casas

Este tutorial irá guiá-lo através de um script Python existente que implementa um modelo de Floresta Aleatória (Random Forest) para prever o preço de venda de casas. O objetivo é entender cada etapa do processo de Machine Learning, desde a configuração inicial até a geração de previsões.

**Contexto**: Este script é típico de um problema de competição no Kaggle, onde você treina um modelo para fazer previsões em um conjunto de dados de teste.

**Pré-requisitos**:


*   Conhecimento básico de Python (variáveis, listas, funções).
*   Familiaridade com conceitos básicos de Machine Learning (treinamento, validação, modelo, previsão).
*   Bibliotecas Python instaladas: pandas, scikit-learn, learntools.

## Etapa 1: Importação de Bibliotecas Essenciais
Aqui, importamos as ferramentas que usaremos para manipulação de dados, treinamento de modelos e avaliação.


In [5]:
# Import helpful libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

Explicação:

1.	**import pandas as pd**: Importa a biblioteca pandas, que é fundamental para trabalhar com dados tabulares (como arquivos CSV). pd é um apelido comum para pandas.
2.	**from sklearn.ensemble import RandomForestRegressor**: Importa o algoritmo RandomForestRegressor da biblioteca scikit-learn. Este é o modelo de Machine Learning que usaremos para fazer as previsões. É um modelo de "floresta aleatória" usado para problemas de regressão (prever um valor contínuo).
3.	**from sklearn.metrics import mean_absolute_error**: Importa a função mean_absolute_error de scikit-learn. Esta métrica será usada para avaliar o desempenho do nosso modelo.
4.	**from sklearn.model_selection import train_test_split**: Importa a função train_test_split de scikit-learn. Esta função é usada para dividir nosso conjunto de dados em partes de treinamento e validação.

## Etapa 2: Carregar os Dados
Nesta etapa, carregaremos os conjuntos de dados de treinamento e teste usando a biblioteca pandas.

In [4]:
import pandas as pd

# Load the training data
train_data_path = '/content/sample_data/california_housing_train.csv'
train_data = pd.read_csv(train_data_path)

# Load the test data
test_data_path = '/content/sample_data/california_housing_test.csv'
test_data = pd.read_csv(test_data_path)

# Display the first few rows of the training data
print("Training Data Head:")
display(train_data.head())

# Display the first few rows of the test data
print("\nTest Data Head:")
display(test_data.head())

Training Data Head:


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0



Test Data Head:


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


**Explicação**:

1.   **from learntools.core import binder**: Importa o módulo binder da biblioteca learntools. Esta biblioteca é frequentemente usada em cursos interativos e notebooks do Kaggle para fornecer feedback instantâneo sobre seu código.
2.   **binder.bind(globals())**: Associa o sistema de verificação de código (learntools) ao ambiente global do seu notebook. Isso permite que as funções de verificação (como step_1.check()) funcionem corretamente.
3.   **from learntools.machine_learning.ex7 import** *: Importa todas as funções de verificação para o Exercício 7 do curso de Machine Learning.
4.   **import os**: Importa o módulo os, que permite interagir com o sistema operacional, como manipular caminhos de arquivos.
5.   **if not os.path.exists("../input/train.csv"): ...**: Este bloco de código verifica se os arquivos train.csv e test.csv já existem no caminho ./input/.
6.   **os.symlink(...)**: Se os arquivos não forem encontrados no caminho ./input/, esta linha cria "links simbólicos" (atalhos) para os arquivos reais que podem estar em um diretório diferente (neste caso, ../input/home-data-for-ml-course/). Isso garante que o script consiga encontrar os dados independentemente de onde eles estejam inicialmente armazenados.


In [7]:
# Load the data, and separate the target
iowa_file_path = train_data_path
home_data = pd.read_csv(iowa_file_path)

## Etapa 3: Definição do Alvo (Target y)
Em problemas de Machine Learning, o "alvo" ou "variável dependente" é o que queremos prever.


In [9]:
# Create target object and call it y
y = home_data.median_house_value
# It is a Serie in Pandas
print(y)

0         66900.0
1         80100.0
2         85700.0
3         73400.0
4         65500.0
           ...   
16995    111400.0
16996     79000.0
16997    103600.0
16998     85800.0
16999     94600.0
Name: median_house_value, Length: 17000, dtype: float64


**Explicação**:

1.	**y =home_data.median_house_value**: Seleciona a coluna Home_data do DataFrame home_data e a atribui à variável y. Home_data é o preço de venda da casa, que é o que nosso modelo tentará prever. y se torna uma Series do pandas.
2.	**print(y)**: Exibe as primeiras e últimas linhas da Series y, mostrando os valores dos preços de venda e confirmando que é uma Series com 1460 entradas.


## Etapa 4: Seleção dos Atributos (Features X)
Os "atributos" ou "features" (variáveis independentes) são as informações que o modelo usará para fazer as previsões.

In [13]:
# Create X (After completing the exercise, you can return to modify this line!)
features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'population', 'households', 'median_income']
# Select columns corresponding to features, and preview the data
X = home_data[features]
# Engineer feature
print(X)

       longitude  latitude  housing_median_age  total_rooms  population  \
0        -114.31     34.19                15.0       5612.0      1015.0   
1        -114.47     34.40                19.0       7650.0      1129.0   
2        -114.56     33.69                17.0        720.0       333.0   
3        -114.57     33.64                14.0       1501.0       515.0   
4        -114.57     33.57                20.0       1454.0       624.0   
...          ...       ...                 ...          ...         ...   
16995    -124.26     40.58                52.0       2217.0       907.0   
16996    -124.27     40.69                36.0       2349.0      1194.0   
16997    -124.30     41.84                17.0       2677.0      1244.0   
16998    -124.30     41.80                19.0       2672.0      1298.0   
16999    -124.35     40.54                52.0       1820.0       806.0   

       households  median_income  
0           472.0         1.4936  
1           463.0         1.8

In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           17000 non-null  float64
 1   latitude            17000 non-null  float64
 2   housing_median_age  17000 non-null  float64
 3   total_rooms         17000 non-null  float64
 4   population          17000 non-null  float64
 5   households          17000 non-null  float64
 6   median_income       17000 non-null  float64
dtypes: float64(7)
memory usage: 929.8 KB


Explicação:

1.	*features = [...]*: Define uma lista chamada features contendo os nomes das colunas que serão usadas como atributos para o modelo. Estas são as informações sobre as casas que o modelo usará para aprender a prever o Home_data.
2.	**X = home_data[features]**: Cria um novo DataFrame chamado X que contém apenas as colunas especificadas na lista features do DataFrame home_data original. X é o nosso conjunto de atributos.
3.	**print(X)**: Exibe as primeiras e últimas linhas do DataFrame X, mostrando os valores dos atributos selecionados.
4.	**X.info()**: Este é um comando muito útil para inspecionar um DataFrame. Ele mostra:

*   O tipo da estrutura (DataFrame).
*   O número de entradas (linhas).
*   O número de colunas e seus nomes.
*   A contagem de valores não-nulos (Non-Null Count) para cada coluna (importante para identificar dados ausentes).
*   O tipo de dado (Dtype) de cada coluna (ex: int64 para números inteiros).
o	O uso de memória.
*   Neste caso, X.info() confirma que temos 7 colunas, 1460 entradas, e todas as colunas são do tipo inteiro e não possuem valores nulos.



## Etapa 5: Divisão dos Dados em Treino e Validação
Para avaliar o desempenho de um modelo de forma justa, dividimos os dados em um conjunto de treinamento (para o modelo aprender) e um conjunto de validação (para testar o modelo em dados que ele nunca viu).


In [15]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
print(train_X.head())
print(val_X.head())
print(train_y.head())
print('\n')
print(val_y.head())

       longitude  latitude  housing_median_age  total_rooms  population  \
12004    -121.39     36.16                28.0       1057.0       288.0   
4646     -118.07     34.07                31.0       1370.0      1062.0   
11486    -121.25     37.97                41.0        855.0       716.0   
15186    -122.27     37.97                10.0      15259.0      7266.0   
635      -117.03     32.63                13.0       2087.0      1165.0   

       households  median_income  
12004       130.0         3.0526  
4646        277.0         3.5156  
11486       206.0         2.0375  
15186      2338.0         6.0666  
635         330.0         5.7789  
       longitude  latitude  housing_median_age  total_rooms  population  \
4752     -118.08     33.85                19.0       4261.0      2621.0   
13348    -121.93     37.73                23.0       2564.0      1043.0   
13459    -121.95     37.32                39.0       1164.0       619.0   
6691     -118.29     34.07             

**Explicação**:

1.	**train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)**:

Esta função divide os dados (X e y) em quatro partes:
 * train_X: Atributos para treinamento.
 * val_X: Atributos para validação.
 * train_y: Alvo para treinamento.
 * val_y: Alvo para validação.
 * random_state=1: Garante que a divisão seja sempre a mesma cada vez que você executa o código. Isso é importante para a reprodutibilidade dos seus resultados.
2.	**print(train_X.head())**, etc.: Exibe as primeiras linhas de cada um dos quatro conjuntos de dados para verificar como a divisão foi feita.


## Etapa 6: Criação e Treinamento do Modelo (Primeira Avaliação)
Aqui, instanciamos nosso modelo de Floresta Aleatória e o "treinamos" usando os dados de treinamento.


In [16]:
# Define a random forest model
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y)

RandomForestRegressor
RandomForestRegressor(random_state=1)

RandomForestRegressor(random_state=1)

**Explicação**:

1.	**rf_model = RandomForestRegressor(random_state=1):** Cria uma instância do modelo RandomForestRegressor e a armazena na variável rf_model.
o	random_state=1: Assim como em train_test_split, define a semente aleatória para garantir que os resultados do treinamento sejam reproduzíveis.
2.	**rf_model.fit(train_X, train_y)**: Esta é a etapa de treinamento do modelo. O modelo rf_model "aprende" os padrões nos dados, ajustando seus parâmetros para melhor mapear os atributos (train_X) aos valores alvo (train_y).


## Etapa 7: Avaliação do Modelo
Após o treinamento, avaliamos o quão bem o modelo se saiu usando os dados de validação.

In [17]:
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 31,067


**Explicação**:

1.	**rf_val_predictions = rf_model.predict(val_X)**: Usa o modelo treinado (rf_model) para fazer previsões nos atributos do conjunto de validação (val_X). Os resultados são armazenados em rf_val_predictions.
2.	**rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)**: Calcula o Mean Absolute Error (MAE) comparando as previsões do modelo (rf_val_predictions) com os valores reais do alvo no conjunto de validação (val_y).

*   MAE: Representa a média das diferenças absolutas entre os valores previstos e os valores reais. Um MAE de 21.857 significa que, em média, o modelo erra o preço de venda em aproximadamente $21.857. Quanto menor o MAE, melhor o modelo.


3.	**print(...)**: Exibe o valor do MAE formatado.


## Etapa 8: Treinamento do Modelo Final em Todos os Dados de Treinamento
Uma vez que você avaliou seu modelo e está satisfeito com seu desempenho, é comum treiná-lo novamente em todos os dados de treinamento disponíveis (não apenas na parte train_X) para que ele aprenda com o máximo de informações possível antes de fazer previsões finais.


In [18]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = RandomForestRegressor()

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(X, y)

RandomForestRegressor
RandomForestRegressor()

RandomForestRegressor()

**Explicação**:


*   **1.	rf_model_on_full_data = RandomForestRegressor()**: Cria uma nova instância do modelo RandomForestRegressor. Desta vez, random_state não foi especificado, o que significa que o comportamento aleatório não será fixo (o que geralmente não é um problema para o modelo final, a menos que você queira reprodutibilidade exata em cada execução).
*   **2.	rf_model_on_full_data.fit(X, y)**: Treina este novo modelo usando todos os dados de atributos (X) e todos os dados de alvo (y) que tínhamos inicialmente no home_data.


## Etapa 9: Preparação dos Dados de Teste e Geração de Previsões Finais
Finalmente, carregamos o conjunto de dados de teste (sem os preços de venda reais), preparamos suas features da mesma forma que fizemos com os dados de treinamento e usamos o modelo final para fazer as previsões que seriam submetidas a uma competição.


In [19]:
# Before submitting, run a check to make sure your test_preds have the right format.
# make predictions which we will submit.

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = test_data[features]

test_preds = rf_model_on_full_data.predict(test_X)

In [20]:
test_preds

array([411346.12, 221130.  , 271440.01, ...,  66833.  , 217787.12,
       498233.95])

**Explicação**:

1.	**test_data_path = '/content/sample_data/california_housing_test.csv'**: Define o caminho para o arquivo de dados de teste.
2.	**test_data = pd.read_csv(test_data_path)**: Carrega os dados de teste em um novo DataFrame chamado test_data.
3.	**test_X = test_data[features]**: Seleciona as mesmas colunas (features) do DataFrame test_data que foram usadas para treinar o modelo. É crucial que as features usadas para prever sejam as mesmas usadas para treinar.
4.	**test_preds = rf_model_on_full_data.predict(test_X)**: Usa o modelo final treinado em todos os dados (rf_model_on_full_data) para fazer previsões nos dados de teste (test_X). Estas são as previsões que você enviaria para a competição.
5.	**step_1.check()**: Executa uma verificação do learntools para garantir que as previsões estejam no formato correto para submissão.
6.	**test_preds**: Exibe o array NumPy contendo as previsões dos preços de venda para cada casa no conjunto de teste.


## Conclusão

*Parabéns*! Você analisou um script Python completo de Machine Learning passo a passo. Você aprendeu como:



*   Configurar o ambiente para o desenvolvimento de ML.
*   Importar bibliotecas essenciais como Pandas e Scikit-learn.
*   Carregar e inspecionar dados.
*   Definir o alvo e os atributos.
*   Dividir os dados para treinamento e validação.
*   Treinar um modelo de Floresta Aleatória.
*   Avaliar o desempenho do modelo usando MAE.
*   Treinar um modelo final e fazer previsões em novos dados.


Este é um fluxo de trabalho fundamental para muitos problemas de Machine Learning. Para melhorar ainda mais este modelo, você poderia explorar:

*   **Engenharia de Atributos (Feature Engineering)**: Criar novas features a partir das existentes (ex: idade da casa a partir de YearBuilt).

*  **Seleção de Atributos**: Experimentar diferentes conjuntos de features.

*  **Ajuste de Hiperparâmetros**: Otimizar as configurações do RandomForestRegressor (ex: n_estimators, max_depth).


*   **Outros Modelos**: Tentar outros algoritmos de ML.
